<a href="https://colab.research.google.com/github/boothmanrylan/historicalFiresQC/blob/main/historicalFiresQC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone Repository and Load Modules

In [ ]:
import tensorflow as tf
import os
from google.colab import drive, auth
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import ee
import glob

drive.mount('/content/drive/')
%cd /content/drive/My\ Drive/historicalFiresQC

auth.authenticate_user()

ee.Authenticate()
ee.Initialize()

# if the directory doesn't exist clone it from github
! if [ ! -d ./historicalFiresQC/ ]; then git clone https://github.com/boothmanrylan/historicalFiresQC.git; fi

% cd ./historicalFiresQC/
! git checkout .
! git pull
%cd ../

! pip install q -r ./historicalFiresQC/requirements.txt

from historicalFiresQC import model as Model
from historicalFiresQC import data as Data
from historicalFiresQC import assessment as Assessment
from historicalFiresQC import visualize as Visualize
from historicalFiresQC import main as Main

# Reload Modules

This is only necessary to run if changes have been made to the historicalFiresQC modules since you started using the colab notebook. It will pull the changes and reload the modules, allowing the changes to be incorporated without having to restart the runtime.

In [ ]:
%cd ./historicalFiresQC
! git pull
%cd ../

from importlib import reload

Model = reload(Model)
Data = reload(Data)
Assessment = reload(Assessment)
Visualize = reload(Visualize)
Main = reload(Main)

# Set Parameters

In [ ]:
params = {
    'bucket':                    'gs://boothmanrylan',         # google cloud storage bucket
    'data_folder':               'historicalFiresQCInputData', # folder inside bucket containing data
    'model_folder':              'historicalFiresQCModels',    # folder inside bucket where models will be saved
    'annotation_type':           'bounding_box',               # how annotations were made: level_slice or bounding_box
    'output':                    'all',                        # what to predict: all, burn_age, burn
    'shape':                     (128, 128),                   # size of 1 input image without bands or batch dimension
    'kernel':                    32,                           # edge buffer around each patch
    'batch_size':                64,                           # Number of images to consider at once
    'stack_image':               False,                        # whether to include the previous image as input to the model
    'include_previous_burn_age': False,                        # whether to include the previous burn age as input to the model
    'burn_age_function':         'scale',                      # function applied to burn age: scale, log, sigmoid, None
    'learning_rate':             1e-4,
    'epochs':                    100,
    'steps_per_epoch':           100,                          # number of batches run through model in one epoch
    'train_model':               False,                        # if false only inference happens 
    'load_model':                True,                         # if true previously trained model weights will be loaded
    'loss_function':             'basic',                      # basic, weigher, or reference_point
    'store_predictions':         True,                         # if true model predictions will be stored in model_folder
}

# Run Model

In [ ]:
output = Main.main(**params)

# Upload Results

In [ ]:
import json

ee_user = 'users/boothmanrylan'
ee_folder = 'historicalFiresQCResults'

all_files = !gsutil ls {output['data_folder']}
mixers = [x for x in all_files if 'json' in x]
mixers.sort()

all_files = !gsutil ls {output['model_folder']}
tfrecords = [x for x in all_files if '.tfrecord' in x]
tfrecords.sort()

model_number = output['model_number']

for i, (t, m) in enumerate(zip(tfrecords, mixers)):
    f = m.replace(output['data_folder'] + '/', '').replace('-mixer.json', '')
    asset_id = os.path.join(ee_user, ee_folder, f'{model_number:05d}', f)
    !earthengine upload image --asset_id={asset_id} {t} {m}